In [1]:
import pandas as pd
from classes.PostgresConnect import PostgresConnect

In [2]:
postgres = PostgresConnect()
postgres.connect()

In [3]:
postgres.sql("sql/questions/reset.sql",file=True)

In [4]:
# requête interactive

region = 'aquitaine' # mettre a jour le contenu ici pour changer de région (i.e: tagne -> Bretagne, compte -> Franche compté etc)

# Ne plus modifier ci-dessous
query = "select d.libelle from departements d join regions r using(reg) where r.ncc LIKE UPPER(%(region)s) or r.libelle LIKE UPPER(%(region)s) order by d.libelle"
params = {'region': f'%{region}%'}

data = postgres.sql(query=query, params=params).all()

print(f'Les départements qui correspondent a la région {region}:')
for dep in data:
    print(f'\t- {dep[0]}')


Les départements qui correspondent a la région aquitaine:
	- Charente
	- Charente-Maritime
	- Corrèze
	- Creuse
	- Deux-Sèvres
	- Dordogne
	- Gironde
	- Haute-Vienne
	- Landes
	- Lot-et-Garonne
	- Pyrénées-Atlantiques
	- Vienne


In [5]:
# requête interactive

commune = 'bordeaux'  # le nom de la ville doit être exact, sans accent ni caractère spéciaux
indicateur = 'pop'  # parmis DECE, LOG, LOGVAC, NAIS, NPER, PMEN, POP, RP, RSECOCC, SUPERF

# Ne plus modifier ci-dessous
data = postgres.sql(
    query="select annee, ncc, dep,valeur from communes c join statistiques s on s.codgeo = c.com where upper(s.indicateur) = upper(%(indicateur)s) and c.ncc = upper( %(commune)s)", 
    params={'commune': commune, 'indicateur': indicateur}
    ).all()

if(len(data)):
    print(f"Information pour l'indicateur {indicateur} pour la commune {commune} ({data[0][2]})")
    for info in data:
        print(f'\t{info[0]}: {info[3]:,}')
else:
    print(f'aucune information pour la commune de {commune} et l\'indicateur {indicateur}')

Information pour l'indicateur pop pour la commune bordeaux (33 )
	1975: 223,131.0
	1968: 266,662.0
	1982: 208,159.0
	2008: 235,891.0
	1990: 210,336.0
	1999: 215,363.0
	2013: 243,626.0
	2018: 257,068.0


In [6]:
# Question 1
data = postgres.sql("sql/questions/question_1/question_1#1.sql",file=True).all()

print('Les 10 villes les plus peuplées de France en 2018:')
for ville in data:
    print(f'\t- {ville[0]}: {ville[1]:,} habitants')


Les 10 villes les plus peuplées de France en 2018:
	- Paris: 2,175,601 habitants
	- Marseille: 868,277 habitants
	- Lyon: 518,635 habitants
	- Toulouse: 486,828 habitants
	- Nice: 341,032 habitants
	- Nantes: 314,138 habitants
	- Montpellier: 290,053 habitants
	- Strasbourg: 284,677 habitants
	- Bordeaux: 257,068 habitants
	- Lille: 233,098 habitants


In [7]:
# Question 2
data = postgres.sql("sql/questions/question_1/question_1#2.sql",file=True).all()

print('Évolution de la population Française au fil des années:')
for population in data:
    population = [0 if v is None else v for v in population]
    print(f'\n\t- Année {population[0]}: {population[1]:,} habitants -> +{population[2]:,} habitants ({population[3]}%)')
    print(f'\t\t- Naissances: {population[4]:,} | Décès: {population[5]:,}')
    if(population[2]):
        print(f'\t\t- Augmentation: {population[2]:,} || Naissances - Décès: {population[4] - population[5]:,} || Différence: {population[2]-(population[4] - population[5]):,}')


Évolution de la population Française au fil des années:

	- Année 1968: 50,798,112 habitants -> +0 habitants (0%)
		- Naissances: 6,149,732 | Décès: 3,909,870

	- Année 1975: 53,764,064 habitants -> +2,965,952 habitants (5.516606780320774%)
		- Naissances: 5,532,080 | Décès: 3,911,267
		- Augmentation: 2,965,952 || Naissances - Décès: 1,620,813 || Différence: 1,345,139

	- Année 1982: 55,569,542 habitants -> +1,805,478 habitants (3.2490424340729676%)
		- Naissances: 6,363,297 | Décès: 4,375,209
		- Augmentation: 1,805,478 || Naissances - Décès: 1,988,088 || Différence: -182,610

	- Année 1990: 58,040,659 habitants -> +2,471,117 habitants (4.257561927406786%)
		- Naissances: 6,877,418 | Décès: 4,819,081
		- Augmentation: 2,471,117 || Naissances - Décès: 2,058,337 || Différence: 412,780

	- Année 1999: 60,149,901 habitants -> +2,109,242 habitants (3.506642513010952%)
		- Naissances: 7,211,899 | Décès: 4,833,385
		- Augmentation: 2,109,242 || Naissances - Décès: 2,378,514 || Différence: -

In [8]:
# Question 3
data = postgres.sql("sql/questions/question_1/question_1#3.sql",file=True).all()

print('Ville la plus peuplée de chaque région en 2018:')
for population in data:
    print(f'\t- Région {population[2]} -> {population[1]} ({population[0]}): {population[3]:,} habitants')


Ville la plus peuplée de chaque région en 2018:
	- Région Île-de-France -> Paris (75056): 2,175,601 habitants
	- Région Provence-Alpes-Côte d'Azur -> Marseille (13055): 868,277 habitants
	- Région Auvergne-Rhône-Alpes -> Lyon (69123): 518,635 habitants
	- Région Occitanie -> Toulouse (31555): 486,828 habitants
	- Région Pays de la Loire -> Nantes (44109): 314,138 habitants
	- Région Grand Est -> Strasbourg (67482): 284,677 habitants
	- Région Nouvelle-Aquitaine -> Bordeaux (33063): 257,068 habitants
	- Région Hauts-de-France -> Lille (59350): 233,098 habitants
	- Région Bretagne -> Rennes (35238): 217,728 habitants
	- Région Normandie -> Le Havre (76351): 169,733 habitants
	- Région Bourgogne-Franche-Comté -> Dijon (21231): 156,854 habitants
	- Région La Réunion -> Saint-Denis (97411): 150,535 habitants
	- Région Centre-Val de Loire -> Tours (37261): 136,463 habitants
	- Région Martinique -> Fort-de-France (97209): 78,126 habitants
	- Région Corse -> Ajaccio (2A004): 70,817 habitants
	

In [9]:
# Création des vues

postgres.sql("sql/questions/question_2/question_2#vue_region.sql",file=True).sql("sql/questions/question_2/question_2#vue_departements.sql",file=True)


In [10]:
# Ajout des colonnes population
data = postgres.sql("sql/questions/question_3/alter_tables.sql",file=True).sql("select ncc,population from regions order by ncc limit 5;").all()


for region in data:
    print(f'{region[0]}: {region[1]}')


AUVERGNE RHONE ALPES: None
BOURGOGNE FRANCHE COMTE: None
BRETAGNE: None
CENTRE VAL DE LOIRE: None
CORSE: None


In [11]:

# Creation et appel de la procédure
data = postgres.sql("sql/questions/question_3/procedure.sql",file=True).sql("call proc_update_population()").sql("select ncc,population from regions order by ncc limit 5;").all()

for region in data:
    print(f'{region[0]}: {region[1]}')


AUVERGNE RHONE ALPES: 7994459
BOURGOGNE FRANCHE COMTE: 2807807
BRETAGNE: 3335414
CENTRE VAL DE LOIRE: 2572853
CORSE: 338554


# Note
Les 2 blocs suivants sont executés dans l'ordre inverse de leur appartition sur l'ennoncé du TP, nous n'avons pas réussi a rendre opérationnel la mise a jour automatiques des populations si les tables régions et départements rejettent toute mise a jour/insertion/suppression.

On aurait pu bloquer les tables au niveau des droits et permettre a la procédure de s'executer mais la question portant sur les trigger nous avons fait le blocage par trigger.

Ainsi pour tester elles s'executent dans l'ordre inverse à la demande:
1. création d'un trigger sur la table de statistiques qui ne se déclenche qu'à la suppression ou si la mise à jour/insertion implique l'indicateur de population
2. mise a jour de la tables statistiques pour augmenter une population (cf: cellule suivante)
3. création des triggers de blocage d'opération
4. test de mise a jour des tables régions et départements afin de constater que l'erreur que nous renvoyons depuis le trigger nous ai bien retournée

In [12]:
# requête interactive

ville = 'bordeaux'  # ville ciblée (se base sur le ncc)
augmentation = 1 # permet de définir de combien l'on augmente la population de la ville donnée

# Mise a jour auto des tables régions et départements quand un indicateur de population est modifié



# ne pas modifier en dessous
sql = """
    select r.libelle,r.population, d.libelle,d.population, c.libelle,valeur::integer, c.com
    from communes c 
    join statistiques s on s.codgeo = c.com 
    join departements d using(dep) 
    join regions r using(reg) 
    where c.ncc = upper(%(ville)s) 
    and s.annee = 2018 
    and s.indicateur = 'POP'
    group by valeur, r.population, d.population, r.libelle, c.libelle, d.libelle, c.com;
""" # ne pas modifier


# dictionnaires utilisés pour constater les différences
pre_update = {'region' : None, 'dep': None, 'ville': None} 
post_update = {'region': None, 'dep': None, 'ville': None}

# ajout du trigger
pre_update = postgres.sql("sql/questions/question_3/triggers_2.sql",file=True).sql(sql, {'ville': ville}).one()
try:
    post_update =postgres.sql(
        query="update statistiques set valeur = %(valeur)s where indicateur = 'POP' and annee = 2018 and codgeo = %(codgeo)s;", 
        params={'valeur': pre_update[-2]+augmentation, 'codgeo': str(pre_update[-1])}
        ).sql(
            query=sql, 
            params={'ville': ville}
            ).one()
    print("Avant mise à jour:")
    print(f'Région: {pre_update[0]} -> {pre_update[1]} habitants')
    print(f'Département: {pre_update[2]} -> {pre_update[3]} habitants')
    print(f'Ville: {pre_update[4]} -> {pre_update[5]} habitants')
    print("\nAprès mise à jour:")
    print(f'Région: {post_update[0]} -> {post_update[1]} habitants')
    print(f'Département: {post_update[2]} -> {post_update[3]} habitants')
    print(f'Ville: {post_update[4]} -> {post_update[5]} habitants')
except Exception as error:
    print(error)


Avant mise à jour:
Région: Nouvelle-Aquitaine -> 5979778 habitants
Département: Gironde -> 1601845 habitants
Ville: Bordeaux -> 257068 habitants

Après mise à jour:
Région: Nouvelle-Aquitaine -> 5979779 habitants
Département: Gironde -> 1601846 habitants
Ville: Bordeaux -> 257069 habitants


In [13]:
# Blocage des tables régions et départements

postgres.sql("sql/questions/question_3/triggers_1.sql",file=True)
try:
    postgres.sql("update regions set population = 10000 where reg = '75';")
except Exception as error:
    print(error)  # UPDATE on regions is not allowed.
try:
    postgres.connect()
    postgres.sql("update departements set population = 10000 where dep = '33';")
except Exception as error:
    print(error)  # UPDATE on departements is not allowed.

postgres.connect()


UPDATE on regions is not allowed.
CONTEXT:  PL/pgSQL function fun_prevent() line 3 at RAISE

UPDATE on departements is not allowed.
CONTEXT:  PL/pgSQL function fun_prevent() line 3 at RAISE



In [14]:
# Plans d'executions

#1
print('#1 - Jointure entre les 2 plus petites tables regions et regions_cheflieu card(18:18)\n')
data = postgres.sql("sql/questions/question_6/explain_small_to_small.sql",file=True).all()
for state in data:
    print(state[0])


#1 - Jointure entre les 2 plus petites tables regions et regions_cheflieu card(18:18)

Hash Join  (cost=12.03..38.07 rows=90 width=896)
  Hash Cond: ((regions_cheflieu.reg)::text = (regions.reg)::text)
  ->  Seq Scan on regions_cheflieu  (cost=0.00..22.70 rows=1270 width=36)
  ->  Hash  (cost=10.90..10.90 rows=90 width=856)
        ->  Seq Scan on regions  (cost=0.00..10.90 rows=90 width=856)


In [15]:
#2
print('#2 - Jointure entre la plus petite table (indicateurs) et la plus grande (statistiques) card(10:2446361)\n')
data = postgres.sql("sql/questions/question_6/explain_smallest_to_biggest.sql",file=True).all()
for state in data:
    print(state[0])


#2 - Jointure entre la plus petite table (indicateurs) et la plus grande (statistiques) card(10:2446361)

Hash Join  (cost=13.15..27048.36 rows=885304 width=670)
  Hash Cond: ((statistiques.indicateur)::text = (indicateurs.code)::text)
  ->  Seq Scan on statistiques  (cost=0.00..24662.04 rows=885304 width=116)
  ->  Hash  (cost=11.40..11.40 rows=140 width=554)
        ->  Seq Scan on indicateurs  (cost=0.00..11.40 rows=140 width=554)


In [16]:

#3
print('#3 - Jointure entre les 2 plus petites tables regions et regions_cheflieu card(18:18) avec tri et ordonnancement\n')
postgres.sql("sql/questions/question_6/explain_small_to_small_ordered.sql",file=True).all()

for state in data:
    print(state[0])

#3 - Jointure entre les 2 plus petites tables regions et regions_cheflieu card(18:18) avec tri et ordonnancement

Hash Join  (cost=13.15..27048.36 rows=885304 width=670)
  Hash Cond: ((statistiques.indicateur)::text = (indicateurs.code)::text)
  ->  Seq Scan on statistiques  (cost=0.00..24662.04 rows=885304 width=116)
  ->  Hash  (cost=11.40..11.40 rows=140 width=554)
        ->  Seq Scan on indicateurs  (cost=0.00..11.40 rows=140 width=554)


In [17]:
#4
print('#4 - Jointure entre la plus petite table (indicateurs) et la plus grande (statistiques) card(10:2446361) avec tri et ordonnancement\n')
data = postgres.sql("sql/questions/question_6/explain_smallest_to_biggest_ordered.sql",file=True).all()

for state in data:
    print(state[0])

#4 - Jointure entre la plus petite table (indicateurs) et la plus grande (statistiques) card(10:2446361) avec tri et ordonnancement

Gather Merge  (cost=24211.10..24211.80 rows=6 width=670)
  Workers Planned: 2
  ->  Sort  (cost=23211.07..23211.08 rows=3 width=670)
        Sort Key: statistiques.valeur
        ->  Nested Loop  (cost=0.14..23211.05 rows=3 width=670)
              ->  Parallel Seq Scan on statistiques  (cost=0.00..23186.53 rows=3 width=116)
                    Filter: (((indicateur)::text = 'POP'::text) AND (annee = 2018) AND ((valeur)::integer > 100000))
              ->  Index Scan using indicateurs_pkey on indicateurs  (cost=0.14..8.16 rows=1 width=554)
                    Index Cond: ((code)::text = 'POP'::text)


## Constats pour les 2eme et 4eme plans d'execution

Par rapport a son homologue (#2) on constate une différence de coût pour la requête (#4), le coût final de cette requête est bien plus petit que celui de la requête qui ne faisait que la jointure (46904.81 contre 38601.06) pour un nombre de ligne très largement inferieur dans cette requête (2446515 contre 9076).

On peut expliquer la différence de coût par le fait que les 2 requêtes ont été executée a la suite lors du test, la première aura peuplé le cache de la base en aillant déjà fait les calculs nécéssaires sur les index de la jointure entre indicateurs et statisitques. La numéro 4 n'étant qu'une extension de la requête 2 (on a juste ajouté des filtres et tri)

On note une grosse différence c'est l'apparition du Workers Planned, qui nous informe qu'en vu d'optimiser le temps de calcul le plannificateur prévois d'utiliser des processus auxiliaires pour executer l'ensemble de la requête (ici 2 processus), s'il intervient ici et n'était pas apparu lors de la #1 c'est certainement a cause du filtre comme on le voit avec le scan parallèle (job auxiliaire) qui applique le filtre.

En général la 2eme requête est bien plus optimisée que celle qui fait uniquement la jointure.


## Comparaison des duo (2-4) et (1-3)
les requêtes #1 et #3 étants effectuées sur de très petites tables (18 entrées) on constate que le plannificateur a tout de même optimisé le temps de calcul pour la #3 qui applique des filtres sur la requête.
La où c'est interessants c'est en comparant la #3 et la #4 qui sont similaire (jointure + filtre) le planificateur n'a pas fait appel a des processus auxiliaires contrairement a la #4, probablement a cause de la taille des tables le plannificateur doit estimer que l'utilisation d'un processus auxiliaire ne ferait pas gagner de temps dans ce cas là.

In [18]:
#5
print('#5 - Multiples jointures entre les tables regions, departements et communes card(18:101:34965)\n')
data = postgres.sql("sql/questions/question_6/explain_triple_join.sql",file=True).all()

for state in data:
    print(state[0])


#5 - Multiples jointures entre les tables regions, departements et communes card(18:101:34965)

Hash Join  (cost=25.29..359.42 rows=1162 width=2604)
  Hash Cond: (communes.dep = (departements.dep)::bpchar)
  ->  Seq Scan on communes  (cost=0.00..312.83 rows=2583 width=876)
  ->  Hash  (cost=24.16..24.16 rows=90 width=1712)
        ->  Hash Join  (cost=12.03..24.16 rows=90 width=1712)
              Hash Cond: ((regions.reg)::bpchar = departements.reg)
              ->  Seq Scan on regions  (cost=0.00..10.90 rows=90 width=856)
              ->  Hash  (cost=10.90..10.90 rows=90 width=868)
                    ->  Seq Scan on departements  (cost=0.00..10.90 rows=90 width=868)


In [19]:
#6
print('#6 - Multiples jointures entre les tables regions, departements et communes card(18:101:34965) mais la jointure est "inversée"\n')
data = postgres.sql("sql/questions/question_6/explain_triple_join_revert.sql", file=True).all()

for state in data:
    print(state[0])


#6 - Multiples jointures entre les tables regions, departements et communes card(18:101:34965) mais la jointure est "inversée"

Hash Join  (cost=25.29..359.42 rows=1162 width=2604)
  Hash Cond: (communes.dep = (departements.dep)::bpchar)
  ->  Seq Scan on communes  (cost=0.00..312.83 rows=2583 width=876)
  ->  Hash  (cost=24.16..24.16 rows=90 width=1708)
        ->  Hash Join  (cost=12.03..24.16 rows=90 width=1708)
              Hash Cond: (departements.reg = (regions.reg)::bpchar)
              ->  Seq Scan on departements  (cost=0.00..10.90 rows=90 width=868)
              ->  Hash  (cost=10.90..10.90 rows=90 width=856)
                    ->  Seq Scan on regions  (cost=0.00..10.90 rows=90 width=856)


## Constat entre la 5 et la 6
La 5 part de régions et joint départements puis communes alors que la 6 part de communes et fini sur régions
On constate que le plannificateur effectue exactements les mêmes requêtes peut importe l'ordre d'appartition des tables dans le select et ses jointures

In [20]:
#7
print('#7 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M)\n')
data = postgres.sql("sql/questions/question_6/explain_quad_join.sql", file=True).all()

for state in data:
    print(state[0])


#7 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M)

Hash Join  (cost=373.95..32338.55 rows=398267 width=2720)
  Hash Cond: ((statistiques.codgeo)::text = (communes.com)::text)
  ->  Seq Scan on statistiques  (cost=0.00..24662.04 rows=885304 width=116)
  ->  Hash  (cost=359.42..359.42 rows=1162 width=2568)
        ->  Hash Join  (cost=25.29..359.42 rows=1162 width=2568)
              Hash Cond: (communes.dep = (departements.dep)::bpchar)
              ->  Seq Scan on communes  (cost=0.00..312.83 rows=2583 width=876)
              ->  Hash  (cost=24.16..24.16 rows=90 width=1708)
                    ->  Hash Join  (cost=12.03..24.16 rows=90 width=1708)
                          Hash Cond: (departements.reg = (regions.reg)::bpchar)
                          ->  Seq Scan on departements  (cost=0.00..10.90 rows=90 width=868)
                          ->  Hash  (cost=10.90..10.90 rows=90 width=856)
                                -

In [21]:
#8
print('#8 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M) avec tri et ordonancement\n')
postgres.sql("sql/questions/question_6/explain_quad_join_ordered.sql", file=True).all()

for state in data:
    print(state[0])


#8 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M) avec tri et ordonancement

Hash Join  (cost=373.95..32338.55 rows=398267 width=2720)
  Hash Cond: ((statistiques.codgeo)::text = (communes.com)::text)
  ->  Seq Scan on statistiques  (cost=0.00..24662.04 rows=885304 width=116)
  ->  Hash  (cost=359.42..359.42 rows=1162 width=2568)
        ->  Hash Join  (cost=25.29..359.42 rows=1162 width=2568)
              Hash Cond: (communes.dep = (departements.dep)::bpchar)
              ->  Seq Scan on communes  (cost=0.00..312.83 rows=2583 width=876)
              ->  Hash  (cost=24.16..24.16 rows=90 width=1708)
                    ->  Hash Join  (cost=12.03..24.16 rows=90 width=1708)
                          Hash Cond: (departements.reg = (regions.reg)::bpchar)
                          ->  Seq Scan on departements  (cost=0.00..10.90 rows=90 width=868)
                          ->  Hash  (cost=10.90..10.90 rows=90 width=856)
       

# Constats entre (7-8) et (4-8)
## 7-8
Comme pour 2-4 on constate que la requête avec des tri est moins coûteuse que la jointure seule également tout ce qui est filtre et ordernancement est géré par des processus auxiliaires (2 également, on peut se demander si ce n'est pas un pour les filtres et un pour le order by, réponse dans la cellule suivante)

## 4-8
Les 2 requêtes sont similaire (le coût dans la branche des workers est très similaire ~30k) et pourtant la 8 consomme plus que la 4, les jointures impactent sont très certainement à mettre en cause bien que d'autres facteurs entrent en jeux (le fait que toutes soient faites a la suite, la machine etc)

In [22]:
#9
print('#9 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M) avec tri\n')
data = postgres.sql("sql/questions/question_6/explain_quad_join_filtered.sql", file=True).all()
for state in data:
    print(state[0])


#9 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M) avec tri

Gather  (cost=1344.74..22033.32 rows=2056 width=2720)
  Workers Planned: 2
  ->  Hash Join  (cost=344.74..20827.72 rows=857 width=2720)
        Hash Cond: (communes.dep = (departements.dep)::bpchar)
        ->  Hash Join  (cost=319.45..20787.11 rows=1857 width=992)
              Hash Cond: ((statistiques.codgeo)::text = (communes.com)::text)
              ->  Parallel Seq Scan on statistiques  (cost=0.00..19497.77 rows=368877 width=116)
              ->  Hash  (cost=319.29..319.29 rows=13 width=876)
                    ->  Seq Scan on communes  (cost=0.00..319.29 rows=13 width=876)
                          Filter: ((ncc)::text ~~ 'B%'::text)
        ->  Hash  (cost=24.16..24.16 rows=90 width=1708)
              ->  Hash Join  (cost=12.03..24.16 rows=90 width=1708)
                    Hash Cond: (departements.reg = (regions.reg)::bpchar)
                    ->  Seq

## constat entre 8 et 9
La seule différence entre les 2 est le order by qui a été enlevé dans la 9, on constate que le plannificateur ne fait plus appel aux processus auxiliaires ce qui peut confirmer l'hypothèse émise précedement

In [23]:
#10
print('#10 - Multiples jointures et fonction d\'aggrégat\n')
data = postgres.sql("sql/questions/question_6/explain_agregate.sql", file=True).all()

for state in data:
    print(state[0])


#10 - Multiples jointures et fonction d'aggrégat

Sort  (cost=10773.82..10773.82 rows=1 width=896)
  Sort Key: groupe_region_population.valeur_max DESC
  ->  Nested Loop  (cost=9435.56..10773.81 rows=1 width=896)
        ->  Nested Loop  (cost=9435.41..10765.63 rows=1 width=930)
              Join Filter: (groupe_region_population.valeur_max = s.valeur)
              ->  Hash Join  (cost=9434.99..9758.80 rows=129 width=892)
                    Hash Cond: (c.dep = (d.dep)::bpchar)
                    ->  Seq Scan on communes c  (cost=0.00..312.83 rows=2583 width=458)
                    ->  Hash  (cost=9434.86..9434.86 rows=10 width=466)
                          ->  Hash Join  (cost=9422.48..9434.86 rows=10 width=466)
                                Hash Cond: (d.reg = groupe_region_population.reg)
                                ->  Hash Join  (cost=12.03..24.16 rows=90 width=462)
                                      Hash Cond: (d.reg = (r.reg)::bpchar)
                              

# constat pour la 10
Le plan étant conséquent nous nous somme servi d'un outil pour analyser les plans d'execution https://explain.dalibo.com/plan/6T4
Dans l'ensemble on constate que le planificateur fait appel a tout ce que l'on a vu précédement, de multiples fois en raison de la sous requête.
L'apparition du groupe HashAggregate est nouveau, il intervient a cause de la fonction d'aggrégat utilisée dans la requête. On constate en revanche que malgré la "complexité" de cette requête son coup est moins elevé que la n°8 alors qu'elle effectue des jointures similaires.

In [24]:
# Index departements

data = postgres.sql("sql/questions/question_7/explain_dep.sql", file=True).all()

for state in data:
    print(state[0])
print('-' * 100)

data = postgres.sql("sql/questions/question_7/explain_dep_order.sql",file=True).all()
for state in data:
    print(state[0])

print('\n'+'#' * 20 + ' CREATION INDEX SUR POPULATION ' + '#'*20)
data = postgres.sql("sql/questions/question_7/create_population_index.sql",file=True).sql("sql/questions/question_7/explain_dep.sql",file=True).all()

for state in data:
    print(state[0])
print('-' * 100)
data = postgres.sql("sql/questions/question_7/explain_dep_order.sql",file=True).all()
for state in data:
    print(state[0])


Seq Scan on departements  (cost=0.00..11.12 rows=30 width=422)
  Filter: (population > 700000)
----------------------------------------------------------------------------------------------------
Sort  (cost=11.86..11.94 rows=30 width=422)
  Sort Key: population
  ->  Seq Scan on departements  (cost=0.00..11.12 rows=30 width=422)
        Filter: (population > 700000)

#################### CREATION INDEX SUR POPULATION ####################
Seq Scan on departements  (cost=0.00..3.26 rows=34 width=422)
  Filter: (population > 700000)
----------------------------------------------------------------------------------------------------
Sort  (cost=4.13..4.21 rows=34 width=422)
  Sort Key: population
  ->  Seq Scan on departements  (cost=0.00..3.26 rows=34 width=422)
        Filter: (population > 700000)


In [25]:
# Test d'index sur la table statistiques

data = postgres.sql("sql/questions/question_7/explain_stats.sql",file=True).all()

print('Sans index avec filtre sans order by:')
for state in data:
    print(state[0])
print('Sans index avec filtre et order by' + ('-' * 100))
data = postgres.sql("sql/questions/question_7/explain_stats_order.sql",file=True).all()
for state in data:
    print(state[0])

print('\n'+'#' * 20 + ' CREATION INDEX SUR valeur ' + '#'*20)
data = postgres.sql("sql/questions/question_7/create_valeur_index.sql",file=True).sql("sql/questions/question_7/explain_stats.sql",file=True).all()

print('Avec index avec filtre sans order by')
for state in data:
    print(state[0])
print('Avec index avec filtre et order by' + ('-' * 100))

data = postgres.sql("sql/questions/question_7/explain_stats_order.sql",file=True).all()

for state in data:
    print(state[0])


Sans index avec filtre sans order by:
Gather  (cost=1000.00..22489.75 rows=1476 width=58)
  Workers Planned: 2
  ->  Parallel Seq Scan on statistiques  (cost=0.00..21342.15 rows=615 width=58)
        Filter: ((valeur > 10000.0) AND ((indicateur)::text = 'POP'::text))
Sans index avec filtre et order by----------------------------------------------------------------------------------------------------
Gather Merge  (cost=22370.66..22514.17 rows=1230 width=58)
  Workers Planned: 2
  ->  Sort  (cost=21370.64..21372.18 rows=615 width=58)
        Sort Key: annee
        ->  Parallel Seq Scan on statistiques  (cost=0.00..21342.15 rows=615 width=58)
              Filter: ((valeur > 10000.0) AND ((indicateur)::text = 'POP'::text))

#################### CREATION INDEX SUR valeur ####################
Avec index avec filtre sans order by
Gather  (cost=1000.00..32506.45 rows=4077 width=58)
  Workers Planned: 2
  ->  Parallel Seq Scan on statistiques  (cost=0.00..31098.75 rows=1699 width=58)
       

# constats pour les index
On constate que sur la table départements, la présence ou non d'un index ne change absolument pas la façon dont le planificateur prépare sa requête probablement a cause de la taille de la table qui est vraiment petite  

En revanche pour la table de statistiques, on constate que le planificateur change d'algorithme et utilise un scan différent (Bitmap heap scan) et qu'en plus de cela il n'a plus recours a des processus auxiliaires (la consommation de resources est très probablement diminuée également de ce fait) et cela a pour impact une reduction drastiques des coûts:
- Entre les requêtes avec filtres et sans order by:
  - diminution du coût estimé par ~2.5 
    - cost=1000.00..32355.95 sans l'index
    - cost=433.27..17392.90 avec index
<hr>

- Entre les requêtes avec filtres et order by:
  - diminution du coût estimé par ~1.8 
    - cost=32152.73..32402.88 sans l'index
    - cost=17538.59..17545.02 avec index

In [26]:
postgres.close(True)